In [36]:
from os import listdir
from os.path import isfile
from fnmatch import fnmatch
from math import isnan, ceil
from pprint import pprint
from statistics import median
from re import match, search

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties

In [37]:
def dump_csv(df, filename):
    df.to_csv(filename, float_format='%.4f')
    return

In [54]:
ratios = ['1:0']
runs = [x for x in range(1, 3 + 1)] 
instances = [x for x in range(1, 2 + 1)]
threads = 1 
clients = [x for x in range(1, 3 + 1)]
vclients = [1, 4, 8, 16, 24, 32]
worker_threads = [8, 16, 32, 64]
mws = [x for x in range(1, 2 + 1)]

mw_multi_index = [ratios, worker_threads]
mw_len_index = len(ratios) * len(worker_threads)

mw_names=['ratio', 'WTS']

mw_index = pd.MultiIndex.from_product(mw_multi_index, names=mw_names)
columns = ['Throughput (ops/s)', 'ResponseTime (ms)']

columns += ['Jobs', 'ServiceTime (ms)', 'WaitingTime (ms)', 'QueueSize']

df = pd.read_csv('../throughput_for_writes/out/middleware_data.csv', index_col=[0,1,2,3,4])
df1 = pd.DataFrame([], index=mw_index, columns=columns)

for ratio in ratios:
    for wt in worker_threads:
        for vc in vclients:
            row = []
            for c in columns:
                if 'ops/s' in c or 'Jobs' in c or 'Queue' in c:
                    row.append(df.unstack().loc[(ratio, wt, vc), c].T.sum().mean())
                else:
                    row.append(df.unstack().loc[(ratio, wt, vc), c].T.mean().mean())
        df1.loc[(ratio, wt)] = row
            
#Service rate is considered to be the max TP for each WT config, since, it constitutes a reasonable lower bound. 

df1['ServiceRate (ops/s)'] = pd.Series()
df1['Rho'] = pd.Series()
df1['En_q'] = pd.Series()
df1['En'] = pd.Series()
df1['Ew'] = pd.Series()

for ratio in ratios:
    for wt in worker_threads:
        df1.loc[(ratio, wt), 'ServiceRate (ops/s)'] = df.unstack().loc[(ratio, wt), 'Throughput (ops/s)'].T.sum().max()
# Utilization
df1['Rho'] = df1['Throughput (ops/s)'] / df1['ServiceRate (ops/s)']
# Expected number of Jobs in the system (w + s)
df1['En'] = df1['Rho'] / (1 - df1['Rho'])
# Expected number of Jobs in the queue(w)
df1['En_q'] = pow(df1['Rho'], 2) / (1 - df1['Rho'])
# Expected AVG Waiting Time in the queue(w)
df1['Ew'] = df1['Rho'] * (1 / df1['ServiceRate (ops/s)']) / (1.0 - df1['Rho']) * 1000
# Expected AVG Response Time
df1['Er'] = (1.0 / (df1['ServiceRate (ops/s)'] * (1.0 - df1['Rho']))) * 1000

dump_csv(df1, 'out/mm1.csv')

          Throughput (ops/s) ResponseTime (ms)    Jobs ServiceTime (ms)  \
ratio WTS                                                                 
1:0   8              7943.05           21.5567  551813          1.90167   
      16             10108.7            16.035  703496             3.01   
      32             12988.6           11.3383  901133          4.59167   
      64             13837.9             9.805  957898          6.89167   

          WaitingTime (ms) QueueSize  ServiceRate (ops/s)       Rho     En_q  \
ratio WTS                                                                      
1:0   8            19.5983   152.027              8043.53  0.987508  78.0662   
      16           12.9583   124.613             10179.31  0.993063  142.163   
      32           6.65333   79.3333             13086.54  0.992517  131.644   
      64              2.76   28.3067             14130.31  0.979308  46.3487   

                En       Ew       Er  
ratio WTS                    